In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

/home/prakash/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
df = pd.read_csv('sample_data_intw.csv')

In [3]:
df.head()

,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,...,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,0,21408I70789,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,1539,...,6.0,0.0,2.0,12,6,0.0,29.000000,29.000000,UPW,2016-07-20
1,1,76462I70374,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,5787,...,12.0,0.0,1.0,12,12,0.0,0.000000,0.000000,UPW,2016-08-10
2,1,17943I70372,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,1539,...,6.0,0.0,1.0,6,6,0.0,0.000000,0.000000,UPW,2016-08-19
3,1,55773I70781,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,947,...,6.0,0.0,2.0,12,6,0.0,0.000000,0.000000,UPW,2016-06-06
4,1,03813I82730,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,2309,...,6.0,0.0,7.0,42,6,0.0,2.333333,2.333333,UPW,2016-06-22


In [4]:
df.set_index(df['pdate'],inplace = True)

df = df.sort_index()

temp = df.drop(df[['pcircle','pdate']],axis = 1)

In [5]:
temp.head()

,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
pdate,,,,,,,,,,,,,,,,,,,,,
2016-06-01,1,07394I84453,1611.0,3.350333,3.350333,104.30,104.30,3.0,0.0,773,...,2,12,6.0,0.5,2.0,12,6,0.5,0.0,0.0
2016-06-01,0,59301I70375,622.0,0.000000,0.000000,11.30,11.30,0.0,0.0,0,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
2016-06-01,0,81671I95201,268.0,0.000000,0.000000,227.83,227.83,0.0,0.0,0,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
2016-06-01,1,05383I91190,204.0,0.621333,0.621333,24.65,24.65,0.0,0.0,0,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
2016-06-01,1,91264I70783,112.0,1.133333,1.133333,5.00,5.00,3.0,0.0,1547,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0


In [6]:
# ax2 = temp.plot.scatter(x='last_rech_amt_ma',
#                        y='daily_decr30',
#                        c='label')

In [7]:

plt.rcParams['figure.figsize'] = (20, 12)
plt.rcParams['font.size'] = 10

# sns.heatmap(temp.corr())

In [8]:
np.corrcoef(temp['last_rech_amt_ma'],temp['medianamnt_loans30'])

array([[1.        , 0.02837036],
       [0.02837036, 1.        ]])

In [9]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209593 entries, 2016-06-01 to 2016-08-21
Data columns (total 34 columns):
label                   209593 non-null int64
msisdn                  209593 non-null object
aon                     209593 non-null float64
daily_decr30            209593 non-null float64
daily_decr90            209593 non-null float64
rental30                209593 non-null float64
rental90                209593 non-null float64
last_rech_date_ma       209593 non-null float64
last_rech_date_da       209593 non-null float64
last_rech_amt_ma        209593 non-null int64
cnt_ma_rech30           209593 non-null int64
fr_ma_rech30            209593 non-null float64
sumamnt_ma_rech30       209593 non-null int64
medianamnt_ma_rech30    209593 non-null float64
medianmarechprebal30    209593 non-null float64
cnt_ma_rech90           209593 non-null int64
fr_ma_rech90            209593 non-null int64
sumamnt_ma_rech90       209593 non-null int64
medianamnt_ma_rech90    209593 n

In [11]:
temp.iloc[:,3:]
# len(temp.aon.unique())

,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,fr_ma_rech30,sumamnt_ma_rech30,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
pdate,,,,,,,,,,,,,,,,,,,,,
2016-06-01,3.350333,3.350333,104.30,104.30,3.0,0.0,773,3,1.0,2319,...,2,12,6.0,0.5,2.0,12,6,0.5,0.0,0.0
2016-06-01,0.000000,0.000000,11.30,11.30,0.0,0.0,0,0,0.0,0,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
2016-06-01,0.000000,0.000000,227.83,227.83,0.0,0.0,0,0,0.0,0,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
2016-06-01,0.621333,0.621333,24.65,24.65,0.0,0.0,0,0,0.0,0,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
2016-06-01,1.133333,1.133333,5.00,5.00,3.0,0.0,1547,1,0.0,1547,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-21,4150.000000,4150.000000,400.26,400.26,1.0,0.0,1539,2,1.0,3078,...,2,24,12.0,0.0,2.0,24,12,0.0,2.0,2.0
2016-08-21,21961.000000,21961.000000,1739.13,1739.13,1.0,0.0,8000,3,1.0,24000,...,2,18,12.0,0.0,2.0,18,12,0.0,1.0,1.0
2016-08-21,340.000000,340.000000,200.13,200.13,1.0,0.0,1539,1,0.0,1539,...,1,6,6.0,0.0,1.0,6,6,0.0,0.0,0.0


In [10]:
temp.describe()

,label,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
count,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,...,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000
mean,0.875177,8112.343445,5381.402289,6082.515068,2692.581910,3483.406534,3755.847800,3712.202921,2064.452797,3.978057,...,2.758981,17.952021,274.658747,0.054029,18.520919,23.645398,6.703134,0.046077,3.398826,4.321485
std,0.330519,75696.082531,9220.623400,10918.812767,4308.586781,5770.461279,53905.892230,53374.833430,2370.786034,4.256090,...,2.554502,17.379741,4245.264648,0.218039,224.797423,26.469861,2.103864,0.200692,8.813729,10.308108
min,0.000000,-48.000000,-93.012667,-93.012667,-23737.140000,-24720.580000,-29.000000,-29.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,246.000000,42.440000,42.692000,280.420000,300.260000,1.000000,0.000000,770.000000,1.000000,...,1.000000,6.000000,6.000000,0.000000,1.000000,6.000000,6.000000,0.000000,0.000000,0.000000
50%,1.000000,527.000000,1469.175667,1500.000000,1083.570000,1334.000000,3.000000,0.000000,1539.000000,3.000000,...,2.000000,12.000000,6.000000,0.000000,2.000000,12.000000,6.000000,0.000000,0.000000,1.666667
75%,1.000000,982.000000,7244.000000,7802.790000,3356.940000,4201.790000,7.000000,0.000000,2309.000000,5.000000,...,4.000000,24.000000,6.000000,0.000000,5.000000,30.000000,6.000000,0.000000,3.750000,4.500000
max,1.000000,999860.755168,265926.000000,320630.000000,198926.110000,200148.110000,998650.377733,999171.809410,55000.000000,203.000000,...,50.000000,306.000000,99864.560864,3.000000,4997.517944,438.000000,12.000000,3.000000,171.500000,171.500000


In [12]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [13]:
# days30_user = temp[['daily_decr30','rental30','sumamnt_ma_rech30','cnt_loans30','payback30','maxamnt_loans30']]

In [14]:
# data_30_medianamtrech = (df['medianamnt_ma_rech30'].loc[(df['pdate'] <= '2016-06-30')])**(1/2)

In [15]:
# # plt.hist(data_30_medianamtrech)
# plt.show()

In [17]:
from scipy.stats import shapiro

In [18]:
min_max_scaler = MinMaxScaler()
np_scaler = min_max_scaler.fit_transform(temp.iloc[:,3:])

In [20]:
Std_scaler = StandardScaler()

In [21]:
std_scaler = Std_scaler.fit_transform(temp.iloc[:,3:])

In [30]:
temp_norm = pd.DataFrame(std_scaler,columns = temp.iloc[:,3:].columns)

In [31]:
temp_norm.head(10)

,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,fr_ma_rech30,sumamnt_ma_rech30,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
0,-0.583265,-0.556762,-0.600728,-0.585588,-0.069619,-0.06955,-0.544737,-0.229802,-0.069652,-0.531136,...,-0.297116,-0.34247,-0.063284,2.045383,-0.073493,-0.439950,-0.334212,2.261798,-0.38563,-0.419233
1,-0.583628,-0.557069,-0.622313,-0.601705,-0.069674,-0.06955,-0.870790,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
2,-0.583628,-0.557069,-0.572057,-0.564181,-0.069674,-0.06955,-0.870790,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
3,-0.583561,-0.557012,-0.619214,-0.599391,-0.069674,-0.06955,-0.870790,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
4,-0.583505,-0.556965,-0.623775,-0.602797,-0.069619,-0.06955,-0.218263,-0.699718,-0.069670,-0.607273,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
5,-0.583310,-0.556800,-0.621962,-0.601443,-0.069674,-0.06955,-0.870790,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
6,-0.583319,-0.556808,-0.623877,-0.602873,-0.069619,-0.06955,-0.218263,0.240114,-0.069670,-0.302330,...,-0.297116,-0.34247,-0.063284,-0.247794,-0.073493,-0.439950,-0.334212,-0.229594,-0.27217,-0.322221
7,-0.583628,-0.557069,-0.624935,-0.603663,-0.069674,-0.06955,-0.870790,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
8,-0.583380,-0.556859,-0.588819,-0.576696,-0.069006,-0.06955,-0.145291,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233
9,-0.583475,-0.556939,-0.611423,-0.593574,-0.068988,-0.06955,-0.218263,-0.934677,-0.069670,-0.759843,...,-0.688582,-0.68770,-0.063284,-0.247794,-0.077941,-0.666624,-0.334212,-0.229594,-0.38563,-0.419233


In [32]:
df_normalized = pd.DataFrame(np_scaler, columns = temp.iloc[:,3:].columns)

In [33]:
df_normalized.describe()

,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,fr_ma_rech30,sumamnt_ma_rech30,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
count,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,...,209593.00000,209593.000000,209593.00000,209593.00000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000,209593.000000
mean,0.020579,0.019255,0.118698,0.125424,0.003790,0.003744,0.037536,0.019596,0.003739,0.009511,...,0.05518,0.058667,0.00275,0.01801,0.003706,0.053985,0.558595,0.015359,0.019818,0.025198
std,0.034662,0.034044,0.019350,0.025661,0.053977,0.053418,0.043105,0.020966,0.053665,0.012517,...,0.05109,0.056797,0.04251,0.07268,0.044982,0.060433,0.175322,0.066897,0.051392,0.060106
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000509,0.000423,0.107865,0.111269,0.000030,0.000029,0.014000,0.004926,0.000000,0.001901,...,0.02000,0.019608,0.00006,0.00000,0.000200,0.013699,0.500000,0.000000,0.000000,0.000000
50%,0.005872,0.004967,0.111472,0.115866,0.000032,0.000029,0.027982,0.014778,0.000002,0.005713,...,0.04000,0.039216,0.00006,0.00000,0.000400,0.027397,0.500000,0.000000,0.000000,0.009718
75%,0.027581,0.024619,0.121682,0.128619,0.000036,0.000029,0.041982,0.024631,0.000006,0.012357,...,0.08000,0.078431,0.00006,0.00000,0.001000,0.068493,0.500000,0.000000,0.021866,0.026239
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [57]:
shapiro(temp_norm.maxamnt_loans30[:100])

(1.0, 1.0)

> ## Feature extracting using PCA

In [36]:
from sklearn.decomposition import PCA

In [37]:
target = df.label

In [39]:
x,y = temp_norm.iloc[:,3:],target

In [40]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1365)

In [41]:
x_train.shape

(146715, 28)

In [42]:
pca = PCA(n_components=15)
X_train = pca.fit_transform(x_train)
X_test = pca.transform(x_test)

In [43]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(max_depth=50, oob_score = True,random_state=2356)
rf_model = rf_classifier.fit(X_train, y_train)

/home/prakash/snap/jupyter/common/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/prakash/snap/jupyter/common/lib/python3.7/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/prakash/snap/jupyter/common/lib/python3.7/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [44]:
y_pred = rf_classifier.predict(X_test)

In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib


cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy', accuracy_score(y_test, y_pred))

[[ 3115  4796]
 [ 2877 52090]]
Accuracy 0.8779700372149242


/home/prakash/snap/jupyter/common/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


> ### Saving the model file

In [46]:
from multiprocessing import Pool,cpu_count
from time import time

In [47]:
param_test1 = {'n_estimators': range(50, 150, 20)}

#best_params is a dict you can pass directly to train a model with optimal settings 
def best_para_2model():
    currtime = time()
    
    grid_search = GridSearchCV(rf_classifier, param_grid=param_test1, cv=10, scoring='f1_macro', n_jobs=4)

    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_ 

    #best_params is a dict you can pass directly to train a model with optimal settings 
    best_model = RandomForestClassifier(param_test1)
    return best_model

In [48]:
result_list = []
def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)

In [51]:
def apply_async_with_callback():
    pool = Pool(processes=4)
    pool.apply_async(best_para_2model, args = (), callback = log_result)
    pool.close()
    pool.join()
    return(result_list)

In [69]:
def save_model(trained_model, model_file_path ):
    """
    :type trained_model: object
    :type model_file_path: str
    """
    from sklearn.externals import joblib
    model = joblib.dump(trained_model, model_file_path)
    return (model_file_path)

In [70]:
save_model(rf_model,'rf_model.pkl')

'rf_model.pkl'